In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages')
sys.path.insert(0,'../')

import sxs
import spherical_functions as sf
import quaternion
import qnmfits

sys.path.insert(0,'../qnmfits')
import read_waveforms


### The `sxs` package lets us easily load SXS waveforms:

In [2]:
h = sxs.load('SXS:BBH:0305/Lev/rhOverM', extrapolation_order=2)
type(h)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping download from 'https://data.black-holes.org/catalog.json' because local file is newer
Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/rhOverM_Asymptotic_GeometricUnits_CoM.h5


sxs.waveforms.waveform_modes.WaveformModes

The returned object is already a `WaveformModes` object, which has lots of handy methods (see the `scri` documentation [here](https://scri.readthedocs.io/en/latest/_autosummary/scri.html#scri.WaveformModes)).

In [3]:
h = read_waveforms.sxs_to_scri_WM(h)

In [4]:
# We can also easily load metadata associated with the simulation
metadata = sxs.load('SXS:BBH:0305/Lev/metadata.json')

Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/metadata.json


In [5]:
M = metadata.remnant_mass
chi_f = metadata.remnant_dimensionless_spin
chi = np.linalg.norm(chi_f)

In [6]:
# If needed, rotate waveform so remnant spin is aligned with z to get best fit 
th_z = np.arccos(chi_f[2]/np.linalg.norm(chi_f))
r_dir = np.cross([0,0,1],chi_f)
r_dir = th_z * r_dir / np.linalg.norm(r_dir)
q = quaternion.from_rotation_vector(-r_dir)
h.rotate_physical_system(q);

# Shift the time array of the WaveformModes object so that zero is at the peak
# Cut off waveform for times greater than 90M 
h_shifted = h.copy()[np.argmin(abs(h.t - 0.)):np.argmin(abs(h.t - 90))+1,:]
h_shifted.t = h_shifted.t - h_shifted.t[0]

### Perform a seven overtone fit to the $h_{22}$ mode using the `qnmfits` package:

In [7]:
qnms = [(2,2,n,1) for n in range(7+1)]
t_0 = 0

best_fit = qnmfits.fit(
    h_shifted,
    qnms,
    M,
    chi,
    t_0=t_0,
    spherical_modes=[(2,2),(3,2)]
)



In [8]:
best_fit

{(2, 2, 0, 1): (0.05679533355205416+0.2337293470023977j),
 (2, 2, 1, 1): (-0.9788414998761575-11.703213016605524j),
 (2, 2, 2, 1): (-1.6161150022651394+139.61196839226312j),
 (2, 2, 3, 1): (9.789989340248885-705.5331343809603j),
 (2, 2, 4, 1): (150.9359124358413+1751.4612726382265j),
 (2, 2, 5, 1): (-451.1276114124514-2119.4344329078376j),
 (2, 2, 6, 1): (398.7522135182466+1232.4070075386346j),
 (2, 2, 7, 1): (-105.74184753267002-287.0537566723667j)}

In [15]:
Q = qnmfits.qnm_modes_as(chi, M, best_fit, h_shifted, t_0=0., t_ref=0.)

In [16]:
qnmfits.mismatch(h_shifted, Q, 0., 90.)

0.7735242842066112